# 2D animation widget

The notebook contains widgets for visualization machine learning methods like sampling, optimization etc.

'AnimationWidget' includes:
* common parameters (control widgets) for all methods, which should not be changed;
* 'AnimationCanvas', where animation is shown and which can be customized;
* 'AnimationExtraWidgets', which control additional parameters (control widgets) for each method and can be customized.

Customizations: '2D_sampling_widget.ipynb' shows how one can sample from the target distribution, those can be defined in the additional notebook "Widget_targets.ipynb", using different sampling methods, those can be defined in "Widget_methods.ipynb".

Other customization of this widget developed for optimization can show, how one can find the minimum (or the maximum) of the target surface using different optimization methods.

In [1]:
# Check existence and versions of required modules
%run config_check.py

In [2]:
# <api>
import time
import collections
import numpy as np
import bqplot
import bqplot.pyplot as bqp
from ipywidgets import widgets, Layout
from IPython import display
from jupyter_cms.loader import load_notebook

In [3]:
# Load targets and methods
trgt = load_notebook('./Widget_targets.ipynb')
mtd = load_notebook('./Widget_methods.ipynb')

## AnimationCanvas

In [4]:
#<api>
class AnimationCanvas(object):
    '''Animated canvas for AnwidgetWidget.'''
    
    def __init__(self, target, title, redraw, start_point=None): 
        self.canvas = bqplot.Figure()
    
        
    def get_canvas(self):
        '''Returns the bqplot.Figure() object to the main widget.'''
        return(self.canvas)
        
    
    def set_data(self, data):
        '''Actualize drawing data (samples/points) from DrawingMethod'''
        pass
        
    def plot_line(self, n):
        '''Plot the first n points of the main line.'''
        pass

## AnimationExtraWidgets

In [5]:
#<api>
class AnimationExtraWidgets(object):
    '''Extra widgets for the dashboard in AnimationWidget.'''

    def __init__(self, method, method_obj, redraw, start_point=None):        
        self.method_obj = method_obj    # DrawingMethod used to create data points
        self.redraw = redraw      # Redraw function of the AnimationWidget object
        self.start_point = start_point
        self.extra_widgets = widgets.VBox([])
        
    def get_widgets(self):
        '''Returns the bqplot.Widget object to the main widget.'''
        return(self.extra_widgets)
    
    def set_data(self, data):
        '''Actualize drawing data (samples/points) from DrawingMethod'''
        pass
        
    def set_counter(self, number):
        '''Actualize counters.'''
        pass
        

## AnimationWidget

In [8]:
#<api>
class AnimationWidget(object):
    '''The main widget, includes inner dashboard, AnimationCanvas, AnimationExtraWidgets, 
    DrawingMethod and Target.'''
    
    def __init__(self, target=None, method=None, save_canvas_mode=False):
        '''General part of widget, that doesn't need to be changed in child classes.
        
        Args:
            target [Target]: a target distribution/surface.
            method [DrawingMethod]: a method for drawing (e.g. sampling) data points.
            save_canvas_mode [bool]: a mode for saving the canvas as .PNG images.
                !!!IMPORTANT!!! It is an experimental function, its work depends on your
                browser settings and the performance of your PC. Be careful, it can quickly
                generate hundreds MB of images.
        
        Quick tutorial for the "save canvas mode":
            0. Set save_canvas_mode=True.
            1. Set your browser settings to saving images without asking the destination folder
            and choose the location.
            2. Choose the start point for saving.
            3. Click on the "Save canvas" checkbox.
            4. !!!IMPORTANT!!! Set the play speed to the minimum (far left).
            5. Press play and wait until you have enough images, then press pause.
            6. (Bonus) If you want to create an animated GIF from these images, we can recommend
            to use ImageMagick for this purpose. You can save all .PNG images in a separate folder,
            go inside it and run a command like this (Linux):
                convert -delay 10 *.png -loop 0 animated_canvas.gif
                
        '''
        
        # Get the dictionaries of methods and targets for this widget
        self.method_dict = self.get_method_dict()
        self.target_dict = self.get_target_dict()
        
        # Set some inner parametes
        self.start_point = None
        self.target = target
        self.choose_target()    # Set the target
        self.size = self.target_obj.get_size()
        
        self.method = method        
        self.choose_method()    # Set the DrawingMethod
        self.save_canvas_mode = save_canvas_mode
        
        # Set variables for canvas's name for saving
        self.redraw_time = time.strftime("%Y-%m-%d_%H-%M-%S")
        self.canvas_num = 0
    
        #### Create the bqplot canvas
        self.canvas_obj = self.get_spec_canvas(self.target_obj, self.title, self.redraw, self.start_point)
        self.canvas = self.canvas_obj.get_canvas()

        #### Create control widgets
        # Widget for selection the method
        self.method_select = widgets.Dropdown(options=self.method_dict, description='Method:', value=self.method)
        self.method_select.observe(self.on_method_select_change, names='value')

        # Widget for selection the target
        self.target_select = widgets.Dropdown(options=self.target_dict, description='Target:', value=self.target)
        self.target_select.observe(self.on_target_select_change, names='value')
        
        # Play slider. !!!Required for the animation. Is shown only in "canvas saving mode".
        self.slider_play = widgets.IntSlider(value=0, min=1, max=self.N, step=1, description='Play', 
                                                 continuous_update=True, readout=False, disable=True)
        self.slider_play.observe(self.on_slider_play_change, names='value')             

        # Slider for the play step (the bigger the step, the quicker the play)
        self.max_interval = 200  # Maximum interval between frames for play widget
        self.slider_play_speed = widgets.IntSlider(value=self.max_interval*0.7, min=1, max=self.max_interval, 
                                                   step=1, description='Play speed', continuous_update=True, 
                                                   readout=False)
        self.slider_play_speed.observe(self.on_slider_play_speed_change, names='value')       
        

        # Redraw button
        self.button_redraw = widgets.Button(description="Redraw")
        self.button_redraw.on_click(self.redraw_function)
            
        # Play/pause/stop buttons widget
        self.step = 1
        self.play = widgets.Play(interval=1, value=1, min=1, max=self.N, step=self.step, description="Press play")
        self.link = widgets.jslink((self.play, 'value'), (self.slider_play, 'value'))   
        
        # Checkbox for "save canvas mode"
        self.save_canvas_checkbox = widgets.Checkbox(value=False, description='Save canvas', disabled=False)
                                                     
        # Create extra widgets
        self.extra_widget_obj = self.get_spec_extra_widgets(self.method, self.method_obj, 
                                                            self.redraw, self.start_point) 
        self.extra_widgets = self.extra_widget_obj.get_widgets()
        
        # Combine all parts of the dashboard
        if self.save_canvas_mode:  # Show more widgets in "save canvas mode"
            self.dashboard = widgets.VBox([self.method_select, 
                                       self.target_select,
                                       widgets.HBox([self.button_redraw, self.play]),
                                       self.slider_play,  
                                       self.slider_play_speed,
                                       self.save_canvas_checkbox])
        else:
            self.dashboard = widgets.VBox([self.method_select, 
                                       self.target_select,
                                       widgets.HBox([self.button_redraw, self.play]),
                                       #self.slider_play,  #Currently is not shown
                                       self.slider_play_speed])
        
        ### Create GUI            
        self.ui = widgets.HBox([self.canvas, widgets.VBox([self.dashboard, self.extra_widgets])])
        self.redraw()
        
        
    def show(self):
        '''Show the widget.'''
        display.display(self.ui) 

################### GET SPECIFIC CANVAS & EXTRA WIDGETS ##############

    def get_spec_canvas(self, target, title, redraw, start_point):
        '''Returns a canvas. Should be redefinde for each specific child class'''
        return(AnimationCanvas(target, title, redraw, start_point))
    
    def get_spec_extra_widgets(self, method, method_obj, redraw, start_point):
        '''Returns extra widgets. Should be redefinde for each specific child class'''
        return(AnimationExtraWidgets(method, method_obj, redraw, start_point))

############################ choose METHOD & TARGET ############################

    def get_method_dict(self):
        '''Returns the methods dictionary. Should be redefined for a specific child class.'''
        method_dict = {}
        return(method_dict)
    
    def get_target_dict(self):
        '''Returns the targets dictionary. Should be redefined for a specific child class.'''
        target_dict = collections.OrderedDict({'Normal':'MN'})
        return(target_dict)


    def choose_method(self):        
        '''choose the drawing method. Needed to be redefined for a specific child class.'''
        self.method_obj = None    # DrawingMethod
        self.N = 1             # Number of data points to produce
        self.step = 1          # Step betwen frames during the animation play
        self.title="Sampling title" 
        
    def choose_target(self): 
        '''Choose the target. Needed to be redefined for a specific child class.'''
        # Set the default target
        if (self.target is None):
            self.target = 'MN'
        # choose the target object for the selected target name
        if (self.target=='MN'):
            self.target_obj = trgt.MultNorm()            
        else:
            raise ValueError('Unknown target.')    
        self.size = self.target_obj.get_size()        
        
                    
############################ REDRAW ############################
        
    def redraw_function(self, button):
        '''Action, when Redraw button is clicked. Just a wrapper for self.redraw().''' 
        self.redraw(self.start_point)

    
    def redraw(self, start_point=None):
        '''Redraws points(samples). Needed to be redefined for a specific child class.'''
        pass
        
############################ SLIDERS ############################ 

    def on_slider_play_change(self, change):
        '''Action on changes of slider_play, drives the animation. Needed to be redefined.'''
        pass       


    def on_slider_play_speed_change(self, change):
        '''Change the animation step/speed.'''
        self.play.interval = self.max_interval - self.slider_play_speed.value + 1
        
            
############################ SELECT MENUS ############################ 

    def on_method_select_change(self, change):
        '''Action on selection a new method.'''

        # Refresh widget settings        
        self.method = self.method_select.value
        display.clear_output(wait=True)       
        
        # This part is similar to initialization 
        self.choose_method()
        self.canvas_obj = self.get_spec_canvas(self.target_obj, self.title, 
                                               self.redraw, self.start_point)
        self.canvas = self.canvas_obj.get_canvas()
        self.extra_widget_obj = self.get_spec_extra_widgets(self.method, self.method_obj, 
                                                            self.redraw, self.start_point) 
        self.extra_widgets = self.extra_widget_obj.get_widgets()
        self.ui = widgets.HBox([self.canvas, widgets.VBox([self.dashboard, self.extra_widgets])])
        
        self.show()
        self.redraw(self.start_point)     

    def on_target_select_change(self, change):
        '''Action on selection a new target.'''
        self.target = self.target_select.value
        self.choose_target()
        self.start_point = None
        self.on_method_select_change(self.method_select.value)
            
    def save_canvas(self, filename='canvas.png'):
        '''Save canvas as PNG.'''
        try:
            self.canvas.save_png(filename)
        except:
            pass
        

## Example

In [9]:
aw = AnimationWidget()
aw.show()